In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("alexander.alexandrov").getOrCreate()
sc = spark.sparkContext

spark

In [3]:
!hdfs dfs -ls /labs/lab06data/ml-100k

Found 23 items
-rw-r--r--   2 hdfs hdfs       6403 2020-09-30 12:16 /labs/lab06data/ml-100k/README
-rw-r--r--   2 hdfs hdfs        716 2020-09-30 12:16 /labs/lab06data/ml-100k/allbut.pl
-rw-r--r--   2 hdfs hdfs        643 2020-09-30 12:16 /labs/lab06data/ml-100k/mku.sh
-rw-r--r--   2 hdfs hdfs    1979173 2020-09-30 12:16 /labs/lab06data/ml-100k/u.data
-rw-r--r--   2 hdfs hdfs        202 2020-09-30 12:16 /labs/lab06data/ml-100k/u.genre
-rw-r--r--   2 hdfs hdfs         36 2020-09-30 12:16 /labs/lab06data/ml-100k/u.info
-rw-r--r--   2 hdfs hdfs     236344 2020-09-30 12:16 /labs/lab06data/ml-100k/u.item
-rw-r--r--   2 hdfs hdfs        193 2020-09-30 12:16 /labs/lab06data/ml-100k/u.occupation
-rw-r--r--   2 hdfs hdfs      22628 2020-09-30 12:16 /labs/lab06data/ml-100k/u.user
-rw-r--r--   2 hdfs hdfs    1586544 2020-09-30 12:16 /labs/lab06data/ml-100k/u1.base
-rw-r--r--   2 hdfs hdfs     392629 2020-09-30 12:16 /labs/lab06data/ml-100k/u1.test
-rw-r--r--   2 hdfs hdfs    1583948 2020-09-30 12

In [8]:
columns = ["user_id", "item_id", "rating", "timestamp"]
udata = spark \
    .read \
    .format("csv") \
    .option("sep", "\t") \
    .option("header", "false") \
    .load("/labs/lab06data/ml-100k/u.data") \
    .toDF(*columns)

In [10]:
udata.show(10)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
|    253|    465|     5|891628467|
|    305|    451|     3|886324817|
|      6|     86|     3|883603013|
+-------+-------+------+---------+
only showing top 10 rows



In [21]:
from pyspark.sql.functions import count

film_id = 286

hist_film = udata \
    .where(f"item_id = {film_id}") \
    .groupby("rating") \
    .agg(count("user_id").alias("cnt")) \
    .orderBy("rating") \
    .collect()

In [22]:
hist_film

[Row(rating='1', cnt=30),
 Row(rating='2', cnt=53),
 Row(rating='3', cnt=100),
 Row(rating='4', cnt=167),
 Row(rating='5', cnt=131)]

In [23]:
hist_all = udata \
    .groupby("rating") \
    .agg(count("user_id").alias("cnt")) \
    .orderBy("rating") \
    .collect()

In [24]:
hist_all

[Row(rating='1', cnt=6110),
 Row(rating='2', cnt=11370),
 Row(rating='3', cnt=27145),
 Row(rating='4', cnt=34174),
 Row(rating='5', cnt=21201)]

In [31]:
import json

output = {
    'hist_film' : [row['cnt'] for row in hist_film],
    'hist_all' : [row['cnt'] for row in hist_all],
}

with open('/data/home/alexander.alexandrov/lab06.json', 'w') as outfile:
    json.dump(output, outfile)

In [33]:
!cat ~/lab06.json

{"hist_film": [30, 53, 100, 167, 131], "hist_all": [6110, 11370, 27145, 34174, 21201]}

In [34]:
spark.stop()

# Review

In [ ]:
Читаемая и удобная реализация лабы.
Более эталонный вариант, чем у меня.